# Compute maximum expected magnitude 
This notebook computes the maximum expected magnitude assuming that the maximum expected magnitude corresponds to the maximum observed magnitude plus a delta.

In [ ]:
%matplotlib inline
import os
import re
import sys
import numpy
import pickle
from prettytable import PrettyTable
from openquake.mbt.oqt_project import OQtProject

## Default settings
If you want to change the ID of the source to be processed you can use [this](./../project/project_set_params_gui.ipynb) notebook.

In [ ]:
project_pickle_filename = os.environ.get('OQMBT_PROJECT')
oqtkp = OQtProject.load_from_file(project_pickle_filename)
oqtkp.directory = os.path.dirname(project_pickle_filename)
model_id = oqtkp.active_model_id
model = oqtkp.models[model_id]
compl_hdf5_filename = oqtkp.compl_hdf5_filename
try:
    area_source_ids_list = getattr(oqtkp,'active_source_id')
except:
    print('Active source ID not defined in the OQMBT project')
    area_source_ids_list = ['10']
print('Processing area source with ID:', area_source_ids_list)
#
# get default max magnitude increment
magnitude_delta = model.magnitude_max_delta
#
# info    
print('Processing model with ID:', model_id)
print('Processing area source with ID:', area_source_ids_list)

## Mmax observed and assigned
The table below describes the values of the maximum magnitude observed in each area source included in the current model and the values of maximum magnitude assigned. 

In [ ]:
# Set table
p = PrettyTable(["ID","mmax obs", "mmax assigned"])
p.align["Source ID"] = 'l'
p.align["mmax obs"] = 'r'
p.align["mmax assigned"] = 'r'
#
for key in sorted(model.sources):
    src = model.sources[key]
    if src.source_type == 'AreaSource':
        alab = ''
        blab = ''
        if hasattr(src, 'mmax_obs'):
            alab = '%6.2f' % (src.mmax_obs)
        if hasattr(src, 'mmax_expected'):
            blab = '%6.2f' % (src.mmax_expected)    
        p.add_row([key, alab, blab])
print(p)

## Read catalogue

In [ ]:
pickle_filename = oqtkp.models[model_id].declustered_catalogue_pickle_filename
pickle_filename = os.path.join(oqtkp.directory, pickle_filename)
fin = open(pickle_filename, 'rb') 
catalogue = pickle.load(fin)
fin.close()

### Create catalogue for the selected areas

In [ ]:
from openquake.mbt.tools.area import create_catalogue
fcatal = create_catalogue(model, catalogue, area_source_ids_list)

## Computing maximum expected magnitude

In [ ]:
mmax_obs = numpy.amax(fcatal.data['magnitude'])
mmax_exp = mmax_obs + magnitude_delta
print('The maximum observed magnitude for this source is        : %.2f' % (mmax_obs))
print('The maximum expected magnitude assigned to this source is: %.2f' % (mmax_exp))

### Update the project info

In [ ]:
for srcid in area_source_ids_list: 
    src = model.sources[srcid]
    src.mmax_obs = mmax_obs
    src.mmax_expected = mmax_exp
    model.sources[srcid] = src
    print(srcid, src.mmax_expected)
oqtkp.models[model_id] = model
oqtkp.save()